In [2]:
# Bibliotecas essênciais:
import pandas as pd
import numpy as np

# Ferramentas de API's:
import requests
import yfinance as yf

# Ferramenta para redimensionar valores (rescaling):
from mlxtend.preprocessing import minmax_scaling

# Ferramentas de plotagem:
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

In [3]:
response = requests.get("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")
response

<Response [200]>

In [4]:
# Criando o dataframe:
response = response.content.decode("utf-8")
result = [x.split(',') for x in response.split('\n')]
df = pd.DataFrame(result[1:], columns= result[0])

In [5]:
# Substituindo os valores vazios por 0:
df = df.replace("", 0, regex=True)

In [6]:
#  Removendo colunas com valores constantes:
df.drop(["country", "city"], axis=1, inplace=True)
df = df[:-1]

In [7]:
# Transformando a coluna de data para o formato apropriado:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

In [8]:
# Como a maioria das colunas está em um formato numérico, vamos converter elas para float:
for column in df.columns:
    try:
        df[column] = df[column].astype(float)
    except:
        continue

In [9]:
# Separando os dados a nível nacional:
df_total = df[df["state"]=="TOTAL"]

In [10]:
# Criando um sub-dataframe para análise de variações de novos casos:
days = pd.DataFrame({
    "newDeaths": [row for row in df_total["newDeaths"]],
    "newCases" : [row for row in df_total["newCases"]],
    "date"     : [row for row in df_total["date"]]
    
})

In [11]:
"""
FUNÇÃO PARA FAZER A AGREGAÇÃO DE DADOS POR MÊS E POR SEMANA:
"""
months = days.set_index("date").resample('M').sum()
display(months.head())

weeks = days.set_index("date").resample('W').sum()

"""
Fazendo essa agregação não podemos trabalhar com colunas de valores agregados!!!! 
Por exemplo, a coluna 'deaths' soma diversas vezes a coluna 'newDeaths', 
e quando ela é agregada ela soma esses valores mais de uma vez!

Isso gera um número absurdo de casos, além de dados errados!

Recomendo usar as colunas newDeaths e newCases da base de dados original,
as outras tem que ser avaliadas antes de serem usadas!!
""";

,newDeaths,newCases
date,,
2020-02-29,0.0,2.0
2020-03-31,202.0,5822.0
2020-04-30,5778.0,81297.0
2020-05-31,23385.0,428975.0
2020-06-30,30416.0,896462.0


In [12]:
"""
Ao invés de usar resample poderiamos pensar em usar groupby, mas isso dificultaria - muito - a plotagem.

Groupby com multy index:
monthly_deaths = df_total.groupby([df_total.date.dt.year, df_total.date.dt.month])['newDeaths'].sum()

Groupby com index resetado:
monthly_deaths = df_total.groupby([df_total.date.dt.year, df_total.date.dt.month], as_index=False)['newDeaths'].sum()


""";

In [13]:
# Reescalando os dados, facilitar a plotagem:
scaled_months = minmax_scaling(months, columns = ["newCases","newDeaths"])
scaled_months.head()

,newCases,newDeaths
date,,
2020-02-29,0.000000,0.000000
2020-03-31,0.001751,0.002453
2020-04-30,0.024465,0.070152
2020-05-31,0.129094,0.283923
2020-06-30,0.269778,0.369288


In [14]:
# Plotando:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x = scaled_months.index,
                               y = scaled_months["newCases"], 
                               name = "Infecções"))

fig = fig.add_trace(go.Bar(x = scaled_months.index,
                           y = scaled_months["newDeaths"], 
                           name = "Óbitos"))

fig.add_vline(x='2021-01-17', line_width=3, line_dash="dash")

fig.add_annotation(x="2021-01-17", y = 1,
            text="Início das Vacinações no Brasil",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor= "#FFFFFF")

fig.update_layout(
    title='Relação das Infecções e Óbitos por Covid'
)

fig.show()

In [15]:
# Analisando a variação de novos casos divulgados:
days_var = days.set_index("date").pct_change()
weeks_var = weeks.pct_change()
months_var = months.pct_change()

In [16]:
# Função para remover outliers e reescalar dados:

def cleaned(dirty_df):
    cleaned_df = pd.DataFrame({})
    
    for column in dirty_df.columns:
        outliers = dirty_df[column].between(dirty_df[column].quantile(.05),dirty_df[column].quantile(.90))
        cleaned_df[column] = pd.DataFrame(dirty_df[column].multiply(outliers))

    for column in cleaned_df.columns:
        cleaned_df[column].replace({0:np.nan}, inplace = True)
        
    cleaned_df_scaled = minmax_scaling(cleaned_df, 
                                       columns = cleaned_df.columns)

    return(cleaned_df_scaled)

### Para otimizar o resultado de nossa análise, iremos remover alguns dos dados extremos:

In [17]:
# Criando uma base de dados limpa:
months_cleaned = pd.DataFrame({})

In [18]:
# Identificando os outiliers da base:
outliers = months_var["newCases"].between(months_var["newCases"].quantile(.05),months_var["newCases"].quantile(.9))

# Removendo os outliers da tabela months e adicionando na tabela months_cleaned:
months_cleaned["newCases"] = pd.DataFrame(months_var["newCases"].multiply(outliers))

# Fazendo a mesma coisa para a coluna de "newDeaths":
outliers = months_var["newDeaths"].between(months_var["newDeaths"].quantile(.05),months_var["newDeaths"].quantile(.9))
months_cleaned["newDeaths"] = pd.DataFrame(months_var["newDeaths"].multiply(outliers))

# Removendo valores nulos:
for column in months_cleaned.columns:
    months_cleaned[column].replace({0:np.nan}, inplace = True)

months_cleaned.head()

,newCases,newDeaths
date,,
2020-02-29,NaN,NaN
2020-03-31,NaN,NaN
2020-04-30,NaN,NaN
2020-05-31,NaN,NaN
2020-06-30,1.089777,0.300663


In [19]:
# Fazendo a mesma coisa para os dados semanais:
weeks_cleaned = pd.DataFrame({})

outliers = weeks_var["newCases"].between(weeks_var["newCases"].quantile(.05),weeks_var["newCases"].quantile(.9))
weeks_cleaned["newCases"] = pd.DataFrame(weeks_var["newCases"].multiply(outliers))

outliers = weeks_var["newDeaths"].between(weeks_var["newDeaths"].quantile(.05),weeks_var["newDeaths"].quantile(.9))
weeks_cleaned["newDeaths"] = pd.DataFrame(weeks_var["newDeaths"].multiply(outliers))

for column in weeks_cleaned.columns:
    weeks_cleaned[column].replace({0:np.nan}, inplace = True)

weeks_cleaned.head()

,newCases,newDeaths
date,,
2020-03-01,NaN,NaN
2020-03-08,NaN,NaN
2020-03-15,NaN,NaN
2020-03-22,NaN,NaN
2020-03-29,NaN,NaN


In [20]:
# Boxplot dos valores que restaram na base:
fig = make_subplots(rows=1, cols=4)

fig.add_trace(go.Box(y=months_cleaned["newCases"].dropna(), name = "Novas Infecções Mensais"))
fig.add_trace(go.Box(y=months_cleaned["newDeaths"].dropna(), name = "Novos Óbitos Mensais"))
fig.add_trace(go.Box(y=weeks_cleaned["newCases"].dropna(), name = "Novas Infecções Semanais"))
fig.add_trace(go.Box(y=weeks_cleaned["newDeaths"].dropna(), name = "Novos Óbitos Semanais"))

fig.update_layout(height=600, width=4000, title_text="Box Plot:  Avaliação das Variações nos Casos de Covid")
fig.show()


# Comparação dos Casos Mensais de Covid com o IPCA

Base de dados disponível em:

<https://dadosabertos.bcb.gov.br/dataset/4449-indice-nacional-de-precos-ao-consumidor-amplo-ipca---precos-monitorados---total>

In [21]:
ipca = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.4449/dados?formato=csv")
ipca

<Response [200]>

In [22]:
ipca = ipca.content.decode("utf-8")
ipca = [x.split(';') for x in ipca.split('\r\n')]

In [23]:
# Removendo as aspas ("") prensentes em cada item do dataframe:
for row in range(0, len(ipca)):
    for item in range(0, len(ipca[row])):
        ipca[row][item] = ipca[row][item].replace('\"', "")

In [24]:
ipca_df = pd.DataFrame(ipca[1:], columns= ipca[0])

In [25]:
# Removendo a última linha, que possui valores nulos:
ipca_df = ipca_df[:-1]

In [26]:
# Colocando a data no formato datetime:
ipca_df["data"] = pd.to_datetime(ipca_df["data"], format="%d/%m/%Y")

In [27]:
# Subtraindo um dia da data, de modo a igualar com a data das tabelas do covid:
ipca_df["data"] = ipca_df["data"]-pd.Timedelta(days=1)

In [28]:
# Trocando os pontos por vírgulas:
ipca_df["valor"] = [row.replace(",", ".") for row in ipca_df["valor"]]

In [29]:
# Colocando o valor no formato float:
ipca_df["valor"] = ipca_df["valor"].astype(float)

In [30]:
# Separando um dataframe para analizar o IPCA entre 2020 e os dias atuais:
ipca = ipca_df.loc[ipca_df["data"]>="2020-02-09"]

In [31]:
# Juntando os dados do covid com os dados do IPCA:
months_ipca = pd.merge_ordered(months_cleaned, ipca["valor"], left_on = months_cleaned.index, right_on = ipca["data"])
months_ipca.rename(columns = {"valor":"ipca"}, inplace=True)
months_ipca.head()

,key_0,newCases,newDeaths,ipca
0,2020-02-29,NaN,NaN,-0.22
1,2020-03-31,NaN,NaN,-2.06
2,2020-04-30,NaN,NaN,-1.02
3,2020-05-31,NaN,NaN,0.89
4,2020-06-30,1.089777,0.300663,1.23


In [32]:
# Reescalando os dados, facilitar a plotagem:
scaled_months_ipca = minmax_scaling(months_ipca.set_index("key_0"), 
                                    columns = ["newCases","newDeaths", "ipca"])
scaled_months_ipca.tail()

,newCases,newDeaths,ipca
key_0,,,
2022-07-31,0.358532,0.616432,0.251397
2022-08-31,NaN,0.214038,0.438547
2022-09-30,0.000000,0.000000,0.632682
2022-10-31,0.122034,0.355347,NaN
2022-11-30,0.259792,NaN,NaN


In [33]:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x = scaled_months_ipca.dropna().index,
                               y =  scaled_months_ipca.dropna()["ipca"], 
                               name = "IPCA no Brasil"))

fig = fig.add_trace(go.Scatter(x = scaled_months_ipca.dropna().index,
                               y = scaled_months_ipca.dropna()["newCases"], 
                               name = "Número de Infecções"))

fig = fig.add_trace(go.Scatter(x = scaled_months_ipca.dropna().index,
                               y = scaled_months_ipca.dropna()["newDeaths"], 
                               name = "Número de Óbitos"))

fig.update_layout(
    title='Variações Mensais: IPCA x Casos de Covid'
)

fig.show()

### Analizando a correlação do IPCA com a COVID em um mesmo mê:

In [34]:
months_heatmap = scaled_months_ipca.dropna().corr().round(decimals=2)
heatmap_dict = {'z': months_heatmap.values.tolist(),
                'x': ["Novas Infecções", "Novos Óbitos", "IPCA"],
                'y': ["Novas Infecções", "Novos Óbitos", "IPCA"]}

In [35]:
fig = go.Figure(data=go.Heatmap(heatmap_dict,
                               text = months_heatmap.values.tolist(),
                               texttemplate="%{text}",
                               textfont={"size":30}))

fig.update_layout(showlegend = False,
                  #width = 700, height = 700,
                  autosize = False,
                  title = "Correlações Mensais do IPCA com a Covid",
                  font_size = 20)

fig.update_yaxes(tickangle = 270, linewidth = 5, tickfont_size =18)
fig.update_xaxes(linewidth = 5, tickfont_size =18)

fig.show()

# Comparação dos Casos de Covid com a Bolsa de Valores e o Dolar

Através da biblioteca yfinance

In [36]:
ibov_month = yf.download('^bvsp', start='2020-01-01', end="2022-11-30", interval='1mo')
usd_month = yf.download('USDBRL=X', start='2020-01-01', end="2022-11-30", interval='1mo')

ibov_week = yf.download('^bvsp', start='2020-01-01', end="2022-11-30", interval='1wk')
usd_week = yf.download('USDBRL=X', start='2020-01-01', end="2022-11-30", interval='1wk')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
# Obtendo as variações mensais e semanais:
ibov_mvar = ibov_month.pct_change()
usd_mvar = usd_month.pct_change()

ibov_wvar = ibov_week.pct_change()
usd_wvar = usd_week.pct_change()

In [38]:
# Unindo as informações da Bolsa com o Dataframe do IPCA
bolsa_months = scaled_months_ipca
bolsa_months = pd.DataFrame(bolsa_months.reset_index())

In [39]:
bolsa_months["ibov"] = [row for row in ibov_mvar["Close"][2:]]
bolsa_months["usd"] = [row for row in usd_mvar["Close"][2:]]
bolsa_months.head()

,key_0,newCases,newDeaths,ipca,ibov,usd
0,2020-02-29,NaN,NaN,0.576816,-0.299044,0.161546
1,2020-03-31,NaN,NaN,0.319832,0.102520,0.027365
2,2020-04-30,NaN,NaN,0.465084,0.085671,-0.000469
3,2020-05-31,NaN,NaN,0.731844,0.087560,0.013090
4,2020-06-30,0.86794,0.507875,0.779330,0.082646,-0.045945


In [40]:
# Reescalando o dataframe:
scaled_bolsa_months = minmax_scaling(bolsa_months.set_index("key_0"), 
                                    columns = ["newCases","newDeaths", "ipca", "ibov", "usd"])

In [41]:
# PLotando:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x = scaled_bolsa_months.dropna().index,
                               y =  scaled_bolsa_months.dropna()["ipca"], 
                               name = "IPCA no Brasil"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_months.dropna().index,
                               y = scaled_bolsa_months.dropna()["newCases"], 
                               name = "Número de Infecções"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_months.dropna().index,
                               y = scaled_bolsa_months.dropna()["newDeaths"], 
                               name = "Número de Óbitos"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_months.dropna().index,
                               y = scaled_bolsa_months.dropna()["ibov"], 
                               name = "Bolsa de Valores"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_months.dropna().index,
                               y = scaled_bolsa_months.dropna()["usd"], 
                               name = "Cotações do Dolar"))

fig.update_layout(
    title='Variações Mensais: IBOV, USD, IPCA e Casos de Covid'
)

fig.show()

In [42]:
fig = px.imshow(scaled_bolsa_months.dropna().corr().round(decimals=2),
                text_auto=True,
                title = "Correlações Mensais: IBOV, USD, IPCA e Casos de Covid")
fig.show()

In [43]:
# Construindo o dataframe semana:
bolsa_weeks = weeks_cleaned
bolsa_weeks = pd.DataFrame(bolsa_weeks.reset_index())
bolsa_weeks["ibov"] = [row for row in ibov_wvar["Close"][8:]]
bolsa_weeks["usd"] = [row for row in usd_wvar["Close"][8:]]

# Reescalando:
scaled_bolsa_weeks = minmax_scaling(bolsa_weeks.set_index("date"), 
                                    columns = ["newCases","newDeaths","ibov", "usd"])

ValueError: Length of values (144) does not match length of index (143)

In [ ]:
# PLotando:
fig = go.Figure()

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_weeks.dropna().index,
                               y = scaled_bolsa_weeks.dropna()["newCases"], 
                               name = "Número de Infecções"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_weeks.dropna().index,
                               y = scaled_bolsa_weeks.dropna()["newDeaths"], 
                               name = "Número de Óbitos"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_weeks.dropna().index,
                               y = scaled_bolsa_weeks.dropna()["ibov"], 
                               name = "Bolsa de Valores"))

fig = fig.add_trace(go.Scatter(x = scaled_bolsa_weeks.dropna().index,
                               y = scaled_bolsa_weeks.dropna()["usd"], 
                               name = "Cotação do Dolar"))

fig.update_layout(
    title='Variações Semanais: IBOV, USD, IPCA e Casos de Covid'
)

fig.show()

NameError: name 'scaled_bolsa_weeks' is not defined

In [ ]:
fig = px.imshow(scaled_bolsa_weeks.dropna().corr(), text_auto=True, title = "Correlações Semanais: IBOV, USD e Casos de Covid")
fig.show()

NameError: name 'scaled_bolsa_weeks' is not defined

# Resultados:

### Forte correlação entre os Casos de Covid e as Mortes por Covid;
### Relação negativa média entre os Casos de Covid e o IPCA;
### Forte relação negativa entre a Contação do Dolar e a Bolsa de Valores brasileira;
### Não existe relação aparente entre o IPCA e a Bolsa de Valores, ou entre o IPCA e a Cotação do Dolar;
### Nem a Bolsa de Valores, nem a Cotação do Dolar tiveram qualquer impacto sobre os Casos de Covid.
